In [0]:
import numpy as np
import time

**DATA**

In [0]:
# IRIS
raw_iris = sc.textFile("/FileStore/tables/iris_data.txt")
# BEANS
raw_beans = sc.textFile("/FileStore/tables/Dry_Bean_Dataset-2.csv")

**CODEBASE**

In [0]:
def get_labels(raw_rdd):
    return raw_rdd.map(lambda string : string.split(",")[-1]).distinct().collect()

def clean_data(raw_rdd):
    aux = raw_rdd.map(lambda string: string.split(",")).map(lambda a: (list(map(lambda x: float(x), a[0:-1])), a[-1])).zipWithIndex().map(lambda t: (t[1], t[0]))
    return (aux.map(lambda x: (x[0], x[1][0])), aux.map(lambda x: (x[0], x[1][1])))

print(get_labels(raw_beans))
print(clean_data(raw_beans)[0].take(2))

['SEKER', 'HOROZ', 'BARBUNYA', 'BOMBAY', 'CALI', 'SIRA', 'DERMASON']
[(0, [28395.0, 610.291, 208.1781167, 173.888747, 1.197191424, 0.549812187, 28715.0, 190.1410973, 0.763922518, 0.988855999, 0.958027126, 0.913357755, 0.007331506, 0.003147289, 0.834222388, 0.998723889]), (1, [28734.0, 638.018, 200.5247957, 182.7344194, 1.097356461, 0.411785251, 29172.0, 191.2727505, 0.783968133, 0.984985603, 0.887033637, 0.953860842, 0.006978659, 0.003563624, 0.909850506, 0.998430331])]


In [0]:
aux = clean_data(raw_iris)
data = aux[0]
classification = aux[1]

In [0]:
def distance(P1, P2):
    d = 0
    for i in range(0, len(P1)):
        d += (P1[i] - P2[i])**2
    return d

distance(data.take(2)[0][1], data.take(2)[1][1])

Out[52]: 0.2899999999999997

In [0]:
def sum_vect(P1, P2):
    aux = []
    for i in range(0, len(P1)):
        aux.append(P1[i] + P2[i])
    return aux

sum_vect(data.take(2)[0][1], data.take(2)[1][1])

Out[55]: [10.0, 6.5, 2.8, 0.4]

In [0]:
def substract_vect(P1, P2):
    aux = []
    for i in range(0, len(P1)):
        aux.append(P1[i] - P2[i])
    return aux

substract_vect(data.take(2)[0][1], data.take(2)[1][1])

Out[56]: [0.1999999999999993, 0.5, 0.0, 0.0]

In [0]:
def divide_vect(P1, P2):
    aux = []
    for i in range(0, len(P1)):
        aux.append(P1[i] / P2[i])
    return aux

divide_vect(data.take(2)[0][1], data.take(2)[1][1])

Out[57]: [1.040816326530612, 1.1666666666666667, 1.0, 1.0]

In [0]:
def mult_vect(k, P):
    aux = []
    for i in range(0, len(P)):
        aux.append(k *  P[i])
    return aux 

mult_vect(1/3, data.take(2)[0][1])

Out[60]: [1.6999999999999997,
 1.1666666666666665,
 0.4666666666666666,
 0.06666666666666667]

In [0]:
def init_centroids(data, k):
    return data.takeSample(False, k)
    
init_centroids(data, 3)

Out[65]: [(78, [6.0, 2.9, 4.5, 1.5]),
 (44, [5.1, 3.8, 1.9, 0.4]),
 (29, [4.7, 3.2, 1.6, 0.2])]

In [0]:
def nearest(P, centroids):
    min_index = -1
    minimum = float("inf")
    for i in range(0, len(centroids)):
        d = distance(P, centroids[i])
        if d < minimum:
            minimum = d
            min_index = i
    return min_index

nearest(data.take(2)[0][1], list(map(lambda t : t[1], init_centroids(data, 3))))

Out[72]: 0

In [0]:
def init_centroids_plusplus(data, k):
    centroids = [(0, data.takeSample(False, 1)[0][1])]
    for i in range(1, k):
        aux = data.map(lambda x: (x[0], distance(x[1], centroids[nearest(x[1], list(map(lambda t: t[1], centroids)))][1])))
        total_weight = aux.map(lambda t: t[1]).reduce(lambda x,y: x+y)
        weight_tuples = aux.map(lambda t: (t[0], t[1]/total_weight)).collect()
        tuple_index = np.random.choice(list(map(lambda t: t[0], weight_tuples)), size=1, replace=False, p=list(map(lambda t: t[1], weight_tuples)))[0]
        new_centroid = (i, data.filter(lambda t : t[0]==tuple_index).take(1)[0][1])
        centroids.append(new_centroid)
    return centroids

init_centroids_plusplus(data, 3)

Out[104]: [(0, [5.0, 3.3, 1.4, 0.2]),
 (1, [5.8, 2.7, 5.1, 1.9]),
 (2, [4.9, 2.4, 3.3, 1.0])]

In [0]:
def get_z(data, centroids):
    return data.map(lambda P: (P[0], nearest(P[1], list(map(lambda c: c[1], centroids)))))

get_z(data, init_centroids(data, 3)).map(lambda t: (t[1], t[0])).countByKey()

Out[108]: defaultdict(int, {1: 50, 2: 42, 0: 58})

In [0]:
def get_j(data, z, centroids):
    return z.join(data).map(lambda e: distance(e[1][1], centroids[e[1][0]][1])).reduce(lambda x,y: x+y)

test_centroids = init_centroids(data, 3)
get_j(data, get_z(data, test_centroids), test_centroids)

Out[110]: 521.56

In [0]:
def compute_centroids(data, z):
    return z.join(data).map(lambda e: (e[1][0], (e[1][1], 1))).reduceByKey(lambda e1, e2: (sum_vect(e1[0], e2[0]), e1[1]+e2[1])).map(lambda e: (e[0], mult_vect(1/e[1][1], e[1][0]))).collect()

test_z = get_z(data, init_centroids(data, 3))
compute_centroids(data, test_z)

Out[116]: [(0, [6.584615384615386, 2.990769230769231, 5.36, 1.9030769230769233]),
 (1,
  [5.005999999999999,
   3.4179999999999997,
   1.4640000000000002,
   0.24400000000000005]),
 (2,
  [5.662857142857143,
   2.6514285714285712,
   4.062857142857142,
   1.2542857142857144])]

In [0]:
def k_means(data, k):
    MAX_ITER = 40
    EPS = 0.01
    
    iteration = 0
    centroids = init_centroids(data, k)
    z = get_z(data, centroids)
    j = get_j(data, z, centroids)
    delta = float("inf")
    aux = j
    
    while iteration < MAX_ITER and delta > EPS :
        centroids = compute_centroids(data, z)
        z = get_z(data, centroids)
        j = get_j(data, z, centroids)
        delta = abs((j/aux - 1))
        aux = j
        iteration += 1
    
    return z


def k_means_plusplus(data, k):
    MAX_ITER = 40
    EPS = 0.01
    
    iteration = 0
    centroids = init_centroids_plusplus(data, k)
    z = get_z(data, centroids)
    j = get_j(data, z, centroids)
    delta = float("inf")
    aux = j
    
    while iteration < MAX_ITER and delta > EPS :
        centroids = compute_centroids(data, z)
        z = get_z(data, centroids)
        j = get_j(data, z, centroids)
        delta = abs((j/aux - 1))
        aux = j
        iteration += 1
    
    return z

In [0]:
k_means(data, 3)

Out[120]: PythonRDD[56302] at RDD at PythonRDD.scala:58

In [0]:
k_means_plusplus(data, 3)

Out[122]: PythonRDD[56380] at RDD at PythonRDD.scala:58

In [0]:
def get_min_row(rdd):
    return rdd.reduce(lambda x,y: list(map(lambda t: t[0] if t[0]<t[1] else t[1], zip(x,y))))

def get_max_row(rdd):
    return rdd.reduce(lambda x,y: list(map(lambda t: t[0] if t[0]>t[1] else t[1], zip(x,y))))

def normalize_data(rdd):
    MINS = get_min_row(data.map(lambda x: x[1]))
    MAXS = get_max_row(data.map(lambda x: x[1]))
    return rdd.map(lambda x: (x[0], divide_vect(substract_vect(x[1], MINS), substract_vect(MAXS, MINS))))

print(get_min_row(data.map(lambda x: x[1])))
print(get_max_row(data.map(lambda x: x[1])))
print(data.take(1)[0])
print(normalize_data(data).take(1)[0])

[4.3, 2.0, 1.0, 0.1]
[7.9, 4.4, 6.9, 2.5]
(0, [5.1, 3.5, 1.4, 0.2])
(0, [0.22222222222222213, 0.6249999999999999, 0.06779661016949151, 0.04166666666666667])


In [0]:
def davies_bouldin(data, z):
    centroids = z.join(data).map(lambda e: (e[1][0], (e[1][1], 1))).reduceByKey(lambda e1, e2: (sum_vect(e1[0], e2[0]), e1[1]+e2[1])).map(lambda e: (e[0], mult_vect(1/e[1][1], e[1][0])))
    
    cluster_distances = z.join(data).map(lambda e: e[1]).join(centroids).map(lambda e: (e[0], (distance(e[1][0], e[1][1]), 1))).reduceByKey(lambda e1, e2: (e1[0]+e2[0], e1[1]+e2[1])).map(lambda e: (e[0], e[1][0]/e[1][1]))
           
    aux = centroids.join(cluster_distances).cartesian(centroids.join(cluster_distances)).filter(lambda e: e[0][0]!=e[1][0]).map(lambda e: (e[0][0], (e[0][1][1]+e[1][1][1])/distance(e[0][1][0], e[1][1][0]))).reduceByKey(lambda x,y: x if x>y else y).map(lambda t: (t[1], 1)).reduce(lambda e1,e2: (e1[0]+e2[0], e1[1]+e2[1]))
    
    return (aux[0]/aux[1])

davies_bouldin(data, k_means(data, 3))

Out[151]: 0.8422176344930344

**PERFORMANCE ANALYSIS**

In [0]:
SIZE = 20

In [0]:
# IRIS: NAIVE MODEL/RAW DATA

total_time = 0
total_score = 0
data = clean_data(raw_iris)[0]
for i in range(SIZE):
    start = time.time_ns()
    prediction = k_means(data, 3)
    end = time.time_ns()
    
    exectime = end - start
    total_time += exectime
    score = davies_bouldin(data, prediction)
    total_score += score

print("exec time (mean): ", (total_time / SIZE) * 10**(-9)," s")
print("davies-bouldin score (mean): ", total_score/SIZE)

exec time (mean):  4.769021284850001  s
davies-bouldin score (mean):  0.30003532119620113


In [0]:
# IRIS: NAIVE MODEL/NORMALIZED DATA

total_time = 0
total_score = 0
data = normalize_data(clean_data(raw_iris)[0])
for i in range(SIZE):
    start = time.time_ns()
    prediction = k_means(data, 3)
    end = time.time_ns()
    
    exectime = end - start
    total_time += exectime
    score = davies_bouldin(data, prediction)
    total_score += score

print("exec time (mean): ", (total_time / SIZE) * 10**(-9)," s")
print("davies-bouldin score (mean): ", total_score/SIZE)

exec time (mean):  3.5562430958  s
davies-bouldin score (mean):  0.38552951411789155


In [0]:
# IRIS: KMEANS++ MODEL/NORMALIZED DATA

total_time = 0
total_score = 0
data = normalize_data(clean_data(raw_iris)[0])
for i in range(SIZE):
    start = time.time_ns()
    prediction = k_means_plusplus(data, 3)
    end = time.time_ns()
    
    exectime = end - start
    total_time += exectime
    score = davies_bouldin(data, prediction)
    total_score += score

print("exec time (mean): ", (total_time / SIZE) * 10**(-9)," s")
print("davies-bouldin score (mean): ", total_score/SIZE)

exec time (mean):  4.54985019425  s
davies-bouldin score (mean):  0.32173108238561754


In [0]:
# BEANS: NAIVE MODEL/RAW DATA

total_time = 0
total_score = 0
data = clean_data(raw_beans)[0]
for i in range(SIZE):
    start = time.time_ns()
    prediction = k_means(data, 3)
    end = time.time_ns()
    
    exectime = end - start
    total_time += exectime
    score = davies_bouldin(data, prediction)
    total_score += score

print("exec time (mean): ", (total_time / SIZE) * 10**(-9)," s")
print("davies-bouldin score (mean): ", total_score/SIZE)

exec time (mean):  20.5458893676  s
davies-bouldin score (mean):  0.13585892352434287


In [0]:
# BEANS: NAIVE MODEL/NORMALIZED DATA

total_time = 0
total_score = 0
data = normalize_data(clean_data(raw_iris)[0])
for i in range(SIZE):
    start = time.time_ns()
    prediction = k_means(data, 3)
    end = time.time_ns()
    
    exectime = end - start
    total_time += exectime
    score = davies_bouldin(data, prediction)
    total_score += score

print("exec time (mean): ", (total_time / SIZE) * 10**(-9)," s")
print("davies-bouldin score (mean): ", total_score/SIZE)

exec time (mean):  4.1730394381  s
davies-bouldin score (mean):  0.3317314914847781


In [0]:
# BEANS: KMEANS++ MODEL/NORMALIZED DATA

total_time = 0
total_score = 0
data = normalize_data(clean_data(raw_iris)[0])
for i in range(SIZE):
    start = time.time_ns()
    prediction = k_means(data, 3)
    end = time.time_ns()
    
    exectime = end - start
    total_time += exectime
    score = davies_bouldin(data, prediction)
    total_score += score

print("exec time (mean): ", (total_time / SIZE) * 10**(-9)," s")
print("davies-bouldin score (mean): ", total_score/SIZE)

exec time (mean):  4.93324807525  s
davies-bouldin score (mean):  0.2224065332759578
